# 1.读取文件 + 切割文件

In [2]:
import { load } from "dotenv";

const env = await load();
const process = {
    env
}

In [3]:
import { TextLoader } from "langchain/document_loaders/fs/text";
const loader = new TextLoader("data/qiu.txt");
const docs = await loader.load();

In [ ]:
// 初始化切割类
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 500,
    chunkOverlap: 100,
});
const splitDocs = await splitter.splitDocuments(docs);

console.log(splitDocs[4]);

Document {
  pageContent: "序曲\n" +
    "　　今天是我的生日，直到晚上爸爸妈妈点上了生日蛋糕的蜡烛，我们三个围着十四个小火苗坐下来，我才想起这事。\n" +
    "　　这是个雷雨之夜，整个宇宙似乎是由密集的闪电和我们的小屋组成。当那蓝色的电光闪起时，窗外的雨珠在一瞬间看得清清楚楚，那雨珠似乎凝固了，像密密地挂在天地间的一串串晶莹的水晶。这时我的脑海中就有一个闪念：世界要是那样的也很有意思，你每天一出门，就在那水晶的密帘中走路，它们在你周围发出丁零丁零的响声，只是，这样玲珑剔透的世界，如何经得住那暴烈的雷电呢……世界在我的眼中总和在别人眼中不一样，我总是努力使世界变形，这是我长这么大对自己唯一的认识。\n" +
    "　　暴雨是从傍晚开始的，自那以后闪电和雷声越来越密，开始，每当一道闪电过后，我脑海中一边回忆着刚才窗外那转瞬即逝的水晶世界，一边绷紧头皮等待着那一声炸雷，但现在，闪电太密集了，我已分不出哪声雷属于哪个闪电了。",
  metadata: { source: "data/qiu.txt", loc: { lines: { from: 25, to: 28 } } }
}


In [7]:
console.log(splitDocs[4].pageContent);

序曲
　　今天是我的生日，直到晚上爸爸妈妈点上了生日蛋糕的蜡烛，我们三个围着十四个小火苗坐下来，我才想起这事。
　　这是个雷雨之夜，整个宇宙似乎是由密集的闪电和我们的小屋组成。当那蓝色的电光闪起时，窗外的雨珠在一瞬间看得清清楚楚，那雨珠似乎凝固了，像密密地挂在天地间的一串串晶莹的水晶。这时我的脑海中就有一个闪念：世界要是那样的也很有意思，你每天一出门，就在那水晶的密帘中走路，它们在你周围发出丁零丁零的响声，只是，这样玲珑剔透的世界，如何经得住那暴烈的雷电呢……世界在我的眼中总和在别人眼中不一样，我总是努力使世界变形，这是我长这么大对自己唯一的认识。
　　暴雨是从傍晚开始的，自那以后闪电和雷声越来越密，开始，每当一道闪电过后，我脑海中一边回忆着刚才窗外那转瞬即逝的水晶世界，一边绷紧头皮等待着那一声炸雷，但现在，闪电太密集了，我已分不出哪声雷属于哪个闪电了。


# 2. 使用向量模型，构建RAGChain

In [ ]:
import { OpenAIEmbeddings } from "@langchain/openai";
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const embeddings = new OpenAIEmbeddings();
const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);

const retriever = vectorstore.asRetriever(2);

Error: OpenAI or Azure OpenAI API key not found

# 3. 构建RetriverChain

In [ ]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";

const convertDocsToString = (documents: Document[]): string => {
    return documents.map((document)=> document.pageContent.join("\n"));
};
const contextRetriverChain = RunnableSequence([
    (input) => input.question,
    retriever,
    convertDocsToString
]);

ReferenceError: retriever is not defined

: 

In [ ]:
const result = await contextRetriverChain.invoke({
    question: "..."
});
console.log(result);

# 4. 创建ChatBot: RAG + prompt + model + output转化

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
const TEMPLATE = `
你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

In [ ]:
import { ChatOpenAI } from "@langchin/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";

const model = new ChatOpenAI();

const ragChain = RunnableSequence.form([
    {
        context: contextRetriverChain,
        question: (input)=> input.question,
    },
    prompt,
    model,
    new StringOutputParser()
]);

const answer = await ragChain.invoke({
    question: "什么是球状闪电"
});
console.log(answer);